In [17]:
# import sys; sys.path.append('/rds/general/user/cb2714/home/wwa'); from wwa import *
import xarray as xr, regionmask, numpy as np

xn,xx,yn,yx = [-15,30,35,60]

In [18]:
# load daily data
da = xr.open_dataset("/rds/general/user/cb2714/home/00_WWA_project_folder/ephemeral/EUR_heat/era5_t2m.nc").t2m
da = da.sel(lon = slice(xn,xx), lat = slice(yx,yn))

In [19]:
# mask land only
rm = regionmask.defined_regions.natural_earth_v5_0_0.land_10.mask(da.lon, da.lat)
da = da.where(rm == 0)

# resample to annual
da = da.rolling(time = 5).mean().groupby("time.season")["JJA"].resample(time = "AS").max()

# relabel dates as years
da = da.assign_coords(time = da.time.dt.year).rename(time = "year")

# save the map for easier reconstruction later
da.mean("year").to_netcdf("map-tmplt_tg5x_era5.nc")

# flatten & convert to data.frame
df = da.stack(xy = ["lat", "lon"]).dropna("xy", "all").to_pandas()

# save data.frame as .csv (split into chunks if really large)
ncols = 2500
if df.shape[1] > ncols:
    for i in range(int(np.ceil(df.shape[1] / ncols))):
        df.iloc[:,slice(i*ncols,(i+1)*ncols)].to_csv("flattened/tg5x-flattened_era5_"+str(i+1).rjust(2,"0")+".csv")
else:
    df.to_csv("flattened/tg5x-flattened_era5.csv")